In [154]:
import pandas as pd 
import numpy as np 
from astropy import units as u
from astropy.coordinates import SkyCoord
from get_nir_spec import get_nir_spec
import astropy.units as u 
from astropy.constants import c
from SpectraTools.fit_line import doppler2wave, wave2doppler
import matplotlib.pyplot as plt 

pd.options.display.max_rows = 999

In [13]:
# instrument / telescope 
def get_instr(row):

    if row.INSTR == 'FIRE': return 'FIRE/Magellan'
    if row.INSTR == 'GNIRS': return 'GNIRS/Gemini'
    if row.INSTR == 'ISAAC': return 'ISAAC/VLT'
    if row.INSTR == 'LIRIS': return 'LIRIS/WHT'
    if row.INSTR == 'NIRI': return 'NIRI/Gemini'
    if row.INSTR == 'NIRSPEC': return 'NIRSPEC/Keck'
    if row.INSTR == 'SINF': return 'SINFONI/VLT'
    if row.INSTR == 'SINF_KK': return 'SINFONI/VLT'
    if row.INSTR == 'SOFI_JH': return 'SofI/NTT'
    if row.INSTR == 'SOFI_LC': return 'SofI/NTT'
    if row.INSTR == 'TRIPLE': return 'TRIPLESPEC/Hale'
    if row.INSTR == 'TRIPLE_S15': return 'TRIPLESPEC/ARC'
    if row.INSTR == 'XSHOOT': return 'XSHOOTER/VLT' 



In [53]:
# Calculate velocity resolution 

df = pd.read_csv('/home/lc585/Dropbox/IoA/nirspec/tables/masterlist_liam.csv', index_col=0) 

for idx, row in df.iterrows():
    
    if row.NIR_PATH != 'None':
        
        wav, dw, flux, err = get_nir_spec(row.NIR_PATH, row.INSTR)
        dw = np.diff(np.log10(wav)).mean()

        dv = c.to('km/s') * (1. - 10. ** -dw)
        df.set_value(idx, 'dv_PIXEL', np.around(dv, decimals=0).value)  

        #-------------------------------
        wav_range = '{0:.2f}'.format((wav.min()*u.AA).to(u.um).value) + '-' + \
                    '{0:.2f}'.format((wav.max()*u.AA).to(u.um).value) 
        df.set_value(idx, 'WAV_RANGE', wav_range)  
        
# These two have SDSS spectra attached so wav_range is wrong 
df.loc[df.INSTR == 'FIRE', 'WAV_RANGE'] = '0.80-2.50'
df.loc[df.INSTR == 'TRIPLE_S15', 'WAV_RANGE'] = '0.95-2.46'     


In [15]:
df = pd.read_csv('/home/lc585/Dropbox/IoA/nirspec/tables/masterlist_liam.csv', index_col=0) 
df = df[df.SPEC_NIR != 'None']
# df.reset_index(inplace=True)
# df.sort_values('RA', inplace=True)

cols = ['ID',
        'DATE',
        'RA',
        'DEC',
        'INSTR_BAND',
        'INSTR',
        'z_ICA']
df = df[cols]

c = SkyCoord(ra=df.RA, dec=df.DEC)
df['RA'] = c.ra.to_string(unit=u.hourangle, precision=2, alwayssign=True, pad=True)
df['DEC'] = c.dec.to_string(precision=2, alwayssign=True, pad=True)

df.DATE.replace(to_replace='None', value='YYYY-MM-DD', inplace=True)

df['INSTR'] = df.apply(get_instr, axis=1)

df['WAV_RANGE'] = wav_ranges 

In [124]:
from SpectraTools.mad import mad

In [142]:
df = pd.read_csv('/home/lc585/Dropbox/IoA/nirspec/tables/masterlist_liam.csv', index_col=0) 
df = df[df.SPEC_NIR != 'None']
df = df[~((df.Ha == 0) & (df.Hb == 0) & (df.OIII == 0))]

for idx, row in df.iterrows(): 

# row = df.loc['QSO296']

    wav, dw, flux, err = get_nir_spec(row.NIR_PATH, row.INSTR)
    wav = wav / (1.0 + row.z_IR)

    instr = row.INSTR
    if instr == 'FIRE': from fit_properties_fire import get_line_fit_props
    if instr == 'GNIRS': from fit_properties_gnirs import get_line_fit_props
    if instr == 'ISAAC': from fit_properties_isaac import get_line_fit_props
    if instr == 'LIRIS': from fit_properties_liris import get_line_fit_props
    if instr == 'NIRI': from fit_properties_niri import get_line_fit_props
    if instr == 'NIRSPEC': from fit_properties_nirspec import get_line_fit_props
    if instr == 'SOFI_JH': from fit_properties_sofi_jh import get_line_fit_props
    if instr == 'SOFI_LC': from fit_properties_sofi_lc import get_line_fit_props
    if instr == 'TRIPLE': from fit_properties_triple import get_line_fit_props
    if instr == 'TRIPLE_S15': from fit_properties_triple_shen15 import get_line_fit_props
    if instr == 'XSHOOT': from fit_properties_xshooter import get_line_fit_props
    if instr == 'SINF': from fit_properties_sinfoni import get_line_fit_props
    if instr == 'SINF_KK': from fit_properties_sinfoni_kurk import get_line_fit_props
    q = get_line_fit_props().all_quasars()
    p = q[df.ix[row.name, 'NUM']]    

    snr = np.zeros(4)

    # Ha -----------------------------------

    w0=6564.89*u.AA
    vdat = wave2doppler(wav*u.AA, w0).value

    # index of region for continuum fit 
    continuum_region = p.ha_continuum_region 
    if continuum_region[0].unit == (u.km/u.s):
        continuum_region[0] = doppler2wave(continuum_region[0], w0)
    if continuum_region[1].unit == (u.km/u.s):
        continuum_region[1] = doppler2wave(continuum_region[1], w0)

    blue_mask = (wav < continuum_region[0][0].value) | (wav > continuum_region[0][1].value)
    red_mask = (wav < continuum_region[1][0].value) | (wav > continuum_region[1][1].value) 

    maskout = p.ha_maskout 

    if maskout is not None:

        for item in maskout:

            if maskout.unit == (u.km/u.s):  

                mask = (vdat > item[0].value) & (vdat < item[1].value) 

            elif maskout.unit == u.AA: 

                mask = (wav.value > item[0].value) & (wav < item[1].value)  

        red_mask = red_mask | mask 
        blue_mask = blue_mask | mask 

    red_snr = np.nanmedian(flux[~red_mask] / np.nanstd(flux[~red_mask]))
    blue_snr = np.nanmedian(flux[~blue_mask] / np.nanstd(flux[~blue_mask]))
    snr[:2] = [red_snr, blue_snr]

    # Hb -----------------------------------

    w0=4862.721*u.AA
    vdat = wave2doppler(wav*u.AA, w0).value

    # index of region for continuum fit 
    continuum_region = p.hb_continuum_region 
    if continuum_region[0].unit == (u.km/u.s):
        continuum_region[0] = doppler2wave(continuum_region[0], w0)
    if continuum_region[1].unit == (u.km/u.s):
        continuum_region[1] = doppler2wave(continuum_region[1], w0)

    blue_mask = (wav < continuum_region[0][0].value) | (wav > continuum_region[0][1].value)
    red_mask = (wav < continuum_region[1][0].value) | (wav > continuum_region[1][1].value) 

    maskout = p.hb_maskout 

    if maskout is not None:

        for item in maskout:

            if maskout.unit == (u.km/u.s):  

                mask = (vdat > item[0].value) & (vdat < item[1].value) 

            elif maskout.unit == u.AA: 

                mask = (wav > item[0].value) & (wav < item[1].value)  

        red_mask = red_mask | mask 
        blue_mask = blue_mask | mask 

    red_snr = np.nanmedian(flux[~red_mask]) / mad(flux[~red_mask])
    blue_snr = np.nanmedian(flux[~blue_mask]) / mad(flux[~blue_mask])
    snr[2:] = [red_snr, blue_snr]
    print idx, np.nanmax(snr)




QSO001 9.93530943779
QSO002 11.5882613923
QSO003 15.2237910477
QSO004 22.4647302673
QSO005 13.2006633446
QSO006 9.40297668649
QSO007 27.0670447536
QSO008 12.1618751316
QSO009 45.9178187043
QSO010 13.7632732952
QSO011 8.0730041542
QSO012 13.6208679675
QSO013 11.1946464165
QSO014 17.9695128357
QSO015 25.2806253363
QSO016 18.9802351022
QSO017 48.6362293506
QSO018 7.85072170494
QSO019 10.8401298107
QSO020 13.3964936848
QSO021 12.4405785342
QSO022 28.269447511
QSO023 17.7353897176
QSO024 8.14400842556
QSO025 18.0357190816
QSO026 3.68073988195
QSO027 10.3825607586
QSO028 6.23218488184
QSO029 4.64278623677
QSO032 12.6326512609
QSO033 8.95160536054
QSO034 8.85102498613
QSO035 5.1973730109
QSO036 9.10565469151
QSO037 10.150556619
QSO038 18.6423454843
QSO039 18.0003949164
QSO040 10.9690776278
QSO041 13.1410107652
QSO042 21.9006146168
QSO043 11.2725995726
QSO044 10.5654850669
QSO045 5.76124585106
QSO046 6.78349288319
QSO047 7.09471847793
QSO048 8.04806191283
QSO049 4.20830792093
QSO050 19.3739046

/data/lc585/venv/lib/python2.7/site-packages/ipykernel/__main__.py:64: RuntimeWarning: invalid value encountered in divide


QSO107 36.4754231645
QSO108 5.8235934634


/data/lc585/venv/lib/python2.7/site-packages/ipykernel/__main__.py:63: RuntimeWarning: invalid value encountered in divide


QSO109 10.9576064319
QSO110 22.3671155526
QSO111 13.2126654076
QSO112 12.9217884987
QSO113 16.6976737438
QSO114 8.42083612853
QSO115 8.09087901832
QSO116 8.76346835119
QSO117 3.51950047048
QSO118 8.28282728805
QSO119 12.973738671
QSO120 12.3004609026
QSO121 8.44243977105
QSO122 6.06596710427
QSO123 3.82314212673
QSO124 14.1150076419
QSO125 11.9742606358
QSO126 14.1981498577
QSO127 19.6928458189
QSO128 13.5481564568
QSO129 9.91373894305
QSO130 48.0023323015
QSO131 4.47800474836
QSO132 8.65524573034
QSO133 11.4082900765
QSO134 17.0061728569
QSO135 7.00230578218
QSO136 8.97054923855
QSO137 17.4298342397
QSO138 8.84410770567
QSO139 12.4842306651
QSO140 18.3787808228
QSO141 15.8658722237
QSO142 7.1807701698
QSO143 10.1574969837
QSO144 7.91285115966
QSO145 7.034384222
QSO146 8.40394447687
QSO147 15.9199034495
QSO148 18.3447193647
QSO149 13.0735030884
QSO150 11.272499093
QSO151 19.8107938637
QSO152 9.08827398063
QSO153 10.7665127628
QSO154 16.0199451874
QSO155 7.93611080059
QSO156 9.006436474

/data/lc585/venv/lib/python2.7/site-packages/ipykernel/__main__.py:100: RuntimeWarning: invalid value encountered in double_scalars


QSO340 8.17128977701
QSO341 5.21704128425
QSO342 18.2331516382
QSO343 54.2598146295
QSO344 7.24046960985
QSO345 10.4860268268
QSO346 25.7382773347
QSO347 16.2714408752
QSO348 16.1969585795
QSO349 27.0272017981
QSO350 15.6132360236
QSO351 24.5819452091
QSO352 21.9048825175
QSO353 21.3235672883
QSO354 44.8914086762
QSO355 21.8233793425
QSO356 12.5397100192
QSO357 21.11998907
QSO358 12.990035213
QSO359 14.9265050493
QSO360 22.9616107529
QSO361 19.9494109188
QSO362 16.253887139
QSO363 23.8444418475
QSO364 15.2666704915
QSO365 48.5828510355
QSO366 13.0911131682
QSO367 25.4505959884
QSO368 24.5709536843
QSO370 14.5767924543
QSO371 1.6592666087
QSO372 6.83125901778
QSO373 8.50189047336
QSO374 13.4312891424
QSO375 18.3176260531
QSO376 12.7367203573
QSO377 9.17441519599
QSO378 9.18145571836
QSO379 7.07372630482
QSO380 4.85293047009
QSO381 15.2976082954
QSO382 21.0796771805
QSO383 5.47577268359
QSO384 14.0166519288
QSO385 8.61970704287
QSO386 4.09433410973
QSO387 7.54110887575
QSO388 10.50620828

/data/lc585/venv/lib/python2.7/site-packages/ipykernel/__main__.py:99: RuntimeWarning: invalid value encountered in double_scalars


QSO639 3.03710776678
QSO640 27.1758106361
QSO642 7.62665313364
QSO644 14.432248492
QSO645 5.73609996398


In [144]:
cd ~/thesis/code/chapter02


/home/lc585/thesis/code/chapter02


In [146]:
from make_table import make_table


In [217]:
df = make_table()


QSO460
QSO552
QSO330
QSO290
QSO289
QSO454
QSO459
QSO475
QSO082
QSO107


make_table.py:108: RuntimeWarning: invalid value encountered in divide
  blue_snr = np.nanmedian(flux[~blue_mask] / np.nanstd(flux[~blue_mask]))


QSO476
QSO001
QSO553
QSO554
QSO555
QSO473
QSO478
QSO556
QSO176
QSO340
QSO479
QSO214
QSO557
QSO559
QSO558
QSO469
QSO560
QSO461
QSO561
QSO562
QSO466
QSO564
QSO565
QSO563
QSO272
QSO083
QSO108
QSO335
QSO273
QSO467
QSO414
QSO279


make_table.py:107: RuntimeWarning: invalid value encountered in divide
  red_snr = np.nanmedian(flux[~red_mask] / np.nanstd(flux[~red_mask]))


QSO109
QSO566
QSO567
QSO110
QSO331
QSO058
QSO002
QSO568
QSO569
QSO572
QSO571
QSO570
QSO215
QSO411
QSO573
QSO574
QSO449
QSO576
QSO575
QSO577
QSO003
QSO578
QSO111
QSO480
QSO004
QSO481
QSO581
QSO482
QSO483
QSO297
QSO582
QSO477
QSO456
QSO583
QSO005
QSO317
QSO584
QSO112
QSO113
QSO470
QSO304
QSO303
QSO585
QSO424
QSO455
QSO516
QSO587
QSO342
QSO589
QSO409
QSO588
QSO517
QSO590
QSO114
QSO115
QSO518
QSO462
QSO006
QSO591
QSO415
QSO345
QSO592
QSO417
QSO116
QSO519
QSO484
QSO593
QSO007
QSO485
QSO377
QSO346
QSO347
QSO595
QSO348
QSO408
QSO594
QSO371
QSO406
QSO398
QSO596
QSO597
QSO349
QSO332
QSO387
QSO117
QSO428
QSO486
QSO438
QSO118
QSO119
QSO219
QSO059
QSO032
QSO220
QSO120
QSO433
QSO520
QSO487
QSO489
QSO488
QSO033
QSO121
QSO221
QSO490
QSO390
QSO432
QSO222
QSO008
QSO491
QSO034
QSO035
QSO521
QSO522
QSO492
QSO323
QSO426
QSO523
QSO445
QSO122
QSO036
QSO060
QSO524
QSO009
QSO380
QSO300
QSO299
QSO177
QSO123
QSO525
QSO225
QSO350
QSO037
QSO038
QSO124
QSO125
QSO178
QSO493
QSO126
QSO226
QSO127
QSO227
QSO026
QSO396

make_table.py:143: RuntimeWarning: invalid value encountered in double_scalars
  red_snr = np.nanmedian(flux[~red_mask]) / mad(flux[~red_mask])


QSO639
QSO603
QSO640
QSO604
QSO389
QSO355
QSO139
QSO182
QSO375
QSO234
QSO027
QSO605
QSO140
QSO399
QSO141
QSO016
QSO044
QSO045
QSO635
QSO606
QSO536
QSO356
QSO086
QSO357
QSO316
QSO315
QSO537
QSO503
QSO087
QSO017
QSO442
QSO142
QSO504
QSO392
QSO429
QSO198
QSO607
QSO421
QSO028
QSO358
QSO199
QSO292
QSO538
QSO143
QSO144
QSO642
QSO145
QSO200
QSO088
QSO146
QSO018
QSO441
QSO644
QSO147
QSO201
QSO436
QSO148
QSO359
QSO275
QSO608
QSO609
QSO435
QSO230
QSO203
QSO373
QSO610
QSO539
QSO360
QSO444
QSO204
QSO386
QSO431
QSO611
QSO019
QSO612
QSO376
QSO361
QSO089
QSO613
QSO505
QSO090
QSO362
QSO205
QSO149
QSO614
QSO506
QSO091
QSO363
QSO443
QSO046
QSO047
QSO364
QSO150
QSO235
QSO540
QSO029
QSO048
QSO179
QSO062
QSO278
QSO507
QSO365
QSO151
QSO020
QSO305
QSO306
QSO508
QSO370
QSO092
QSO509
QSO366
QSO541
QSO180
QSO510
QSO367
QSO093
QSO207
QSO615
QSO236
QSO152
QSO368
QSO153
QSO383
QSO208
QSO463
QSO237
QSO049
QSO050
QSO511
QSO154
QSO238
QSO437
QSO440
QSO439
QSO542
QSO155
QSO543
QSO457
QSO393
QSO397
QSO156
QSO512
QSO210

make_table.py:144: RuntimeWarning: invalid value encountered in double_scalars
  blue_snr = np.nanmedian(flux[~blue_mask]) / mad(flux[~blue_mask])


QSO339
QSO621
QSO333
QSO416
QSO329
QSO622
QSO623
QSO624
QSO514
QSO097
QSO171
QSO551
QSO313
QSO314
QSO447
QSO625
QSO626
QSO515
QSO024
QSO453
QSO627
QSO098
QSO309
QSO310
QSO474
QSO025
QSO628
QSO172
QSO629
QSO631
QSO302
QSO301
QSO632
QSO633
QSO173
QSO413
QSO245
QSO344
QSO634
QSO174
QSO175


In [212]:
df



,ID,DATE,RA,DEC,INSTR,WAV_RANGE,dv_PIXEL,SNR,z_OIII,z_Ha,z_Hb
QSO460,J000039-001804,2015-09-02,+00h00m39.00s,-00d18m03.90s,SofI/NTT,1.50-2.54,154.0,4.9,NaN,2.1412,2.1391
QSO552,J000345-232353,2009-07-07,+00h03m45.00s,-23d23m53.40s,SINFONI/VLT,1.44-1.87,36.0,12.7,2.2657,NaN,2.2653
QSO330,J000345-232353,2011-09-18,+00h03m45.00s,-23d23m53.40s,SofI/NTT,1.48-1.83,63.0,36.0,2.2644,NaN,2.2776
QSO290,J000451-084450,2013-07-12,+00h04m50.66s,-08d44m49.63s,XSHOOTER/VLT,0.31-2.28,15.0,10.3,3.0038,NaN,3.0052
QSO289,J000451-084452,2013-08-08,+00h04m50.91s,-08d44m51.98s,XSHOOTER/VLT,0.31-2.28,15.0,5.4,2.9991,NaN,NaN
QSO454,J000500-003348,2015-09-01,+00h05m00.42s,-00d33m48.20s,SofI/NTT,1.50-2.54,154.0,8.2,NaN,2.1842,2.1850
QSO459,J000501+010221,2015-09-02,+00h05m00.53s,+01d02m20.80s,SofI/NTT,1.50-2.54,154.0,6.8,NaN,2.1334,2.1317
QSO475,J001016+001228,2015-09-04,+00h10m16.49s,+00d12m27.60s,SofI/NTT,1.50-2.54,154.0,8.9,NaN,2.2855,2.2828
QSO082,J001247+001239,2013-06-06,+00h12m47.12s,+00d12m39.49s,ISAAC/VLT,1.52-1.60,15.0,19.1,NaN,NaN,2.1618
QSO107,J001708+813508,2012-08-04,+00h17m08.48s,+81d35m08.10s,TRIPLESPEC/Hale,0.94-2.80,39.0,36.5,3.3934,NaN,NaN


In [186]:
df = pd.read_csv('/home/lc585/Dropbox/IoA/nirspec/tables/masterlist_liam.csv', index_col=0) 
df.loc[df['OIII_Z_FLAG'].isin([1, 2]), 'z_OIII'] = np.nan 

In [226]:
print len(df)

462


In [227]:
print df[:15].to_latex(index=False, na_rep='', longtable=True)

\begin{longtable}{llllllrrrrr}
\toprule
             ID &        DATE &             RA &            DEC &            INSTR &  WAV\_RANGE &  dv\_PIXEL &   SNR &  z\_OIII &    z\_Ha &    z\_Hb \\
\midrule
\endhead
\midrule
\multicolumn{3}{r}{{Continued on next page}} \\
\midrule
\endfoot

\bottomrule
\endlastfoot
 J000039-001804 &  2015-09-02 &  +00h00m39.00s &  -00d18m03.90s &         SofI/NTT &  1.50-2.54 &     154.0 &   4.9 &         &  2.1412 &  2.1391 \\
 J000345-232353 &  2009-07-07 &  +00h03m45.00s &  -23d23m53.40s &      SINFONI/VLT &  1.44-1.87 &      36.0 &  12.7 &  2.2657 &         &  2.2653 \\
 J000345-232353 &  2011-09-18 &  +00h03m45.00s &  -23d23m53.40s &         SofI/NTT &  1.48-1.83 &      63.0 &  36.0 &  2.2644 &         &  2.2776 \\
 J000451-084450 &  2013-07-12 &  +00h04m50.66s &  -08d44m49.63s &     XSHOOTER/VLT &  0.31-2.28 &      15.0 &  10.3 &  3.0038 &         &  3.0052 \\
 J000451-084452 &  2013-08-08 &  +00h04m50.91s &  -08d44m51.98s &     XSHOOTER/VLT &  0.31-

In [240]:
df_tmp = df[df.INSTR == 'SofI/NTT']

In [243]:
df_tmp.DATE.sort_values()

QSO408    2011-09-18
QSO330    2011-09-18
QSO329    2011-09-18
QSO424    2011-09-18
QSO337    2011-09-18
QSO409    2011-09-19
QSO332    2011-09-19
QSO331    2011-09-19
QSO400    2011-09-19
QSO340    2011-09-19
QSO338    2011-09-19
QSO339    2011-09-19
QSO333    2011-09-20
QSO335    2011-09-20
QSO342    2011-09-20
QSO334    2011-09-20
QSO417    2011-09-21
QSO416    2011-09-21
QSO344    2011-09-21
QSO411    2011-09-21
QSO410    2011-09-21
QSO415    2011-09-22
QSO413    2011-09-22
QSO412    2011-09-22
QSO336    2011-09-22
QSO343    2011-09-22
QSO362    2012-03-05
QSO360    2012-03-05
QSO357    2012-03-05
QSO354    2012-03-05
QSO352    2012-03-05
QSO368    2012-03-05
QSO345    2012-03-05
QSO348    2012-03-06
QSO355    2012-03-06
QSO361    2012-03-06
QSO363    2012-03-06
QSO370    2012-03-06
QSO423    2012-03-06
QSO358    2012-03-07
QSO356    2012-03-07
QSO350    2012-03-07
QSO346    2012-03-07
QSO359    2012-03-07
QSO373    2012-03-08
QSO372    2012-03-08
QSO365    2012-03-08
QSO349    201